# Justifying a random forest's predictions
## Libraries and experimental data set

In [1]:
# Libraries to be used
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from lime.lime_tabular import LimeTabularExplainer
from scipy.sparse import hstack

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data set for experiments
from sklearn.datasets import load_wine
dataset = load_wine()
X = pd.DataFrame(dataset.data, columns = dataset.feature_names)
y = dataset.target

In [3]:
# Split data into training and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    stratify = y,
    train_size = 100,
    random_state = 2017   # for reproducibility
)

## Exploratory analysis and random forest classifiers

In [4]:
# Take a glance at feature distributions, broken down by class
Xtrain.groupby(ytrain).quantile([.1,.9])

alcohol  malic_acid    ash  alcalinity_of_ash  magnesium  \
0 0.1   13.050       1.538  2.222              14.40       94.2   
  0.9   14.218       3.758  2.700              20.40      119.6   
1 0.1   11.637       0.976  1.813              16.80       83.6   
  0.9   13.032       2.944  2.673              22.61      110.2   
2 0.1   12.486       1.812  2.242              19.10       86.0   
  0.9   13.980       4.654  2.716              25.00      112.4   

       total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0 0.1          2.412       2.414                 0.220            1.368   
  0.9          3.200       3.512                 0.398            2.380   
1 0.1          1.648       1.277                 0.208            0.820   
  0.9          2.905       2.797                 0.532            2.121   
2 0.1          1.292       0.494                 0.266            0.818   
  0.9          2.240       1.292                 0.610            1.684   

       color_intensity    hue  od280/od315_of_diluted_wines  proline  
0 0.1            3.858  0.890                         2.718    782.0  
  0.9            6.760  1.224                         3.578   1285.0  
1 0.1            1.995  0.786                         2.101    340.3  
  0.9            3.991  1.288                         3.300    721.2  
2 0.1            5.200  0.556                         1.330    476.0  
  0.9           10.632  0.750                         1.884    782.0

In [5]:
# Train a random forest classifier for each of the three classes
clf = []
for clas in range(3):
    clf.append(
        RandomForestClassifier(
            n_estimators = 100, n_jobs = -1,
            random_state = 2017   # for reproducibility
        ).fit(Xtrain, ytrain == clas)
    )

In [6]:
# Check the AUC's of the classifiers on the test data
for clas in range(3):
    print(roc_auc_score(ytest == clas, clf[clas].predict_proba(Xtest)[:,1]))

1.0
0.997597803706
1.0


## Explanations from LIME

In [7]:
# Create a LIME explainer for tabular data
explainer = LimeTabularExplainer(
    Xtrain.values, feature_names = Xtrain.columns,
    random_state = 2017   # for reproducibility
)

def explain_row(clf, row, num_reasons = 2):
    '''
    Produce LIME explanations for a single row of data.
        * `clf` is a binary classifier (with a predict_proba method),
        * `row` is a row of features data,
        * `num_reasons` (default 2) is the number of 
          reasons/explanations to be produced.
          
    '''
    exp = [
        exp_pair[0] for exp_pair in     # Get each explanation (a string)
        explainer.explain_instance(     # from the LIME explainer
            row, clf.predict_proba,     # for the given row and classifier
            labels = [1],               # and label 1 ("positives")
            num_features = num_reasons  # for up to `num_reasons` explanations
        ).as_list()
        if exp_pair[1] > 0              # but only for positive explanations 
    ][:num_reasons]
    
    # Fill in any missing explanations with blanks
    exp += [''] * (num_reasons - len(exp))  
    return exp


def predict_explain(rf, X, num_reasons = 2):
    '''
    Produce scores and LIME explanations for every row in a data frame.
        * `rf` is a binary classifier with a predict_proba method,
        * `X` is the features data frame,
        * `num_reasons` (default 2) is the number of 
          reasons/explanations to be produced for each row.
          
    '''
    # Prepare the structure to be returned
    pred_ex = Xtest[[]]
    
    # Get the scores from the classifier
    pred_ex['SCORE'] = rf.predict_proba(X)[:,1]
    
    # Get the reasons/explanations for each row
    cols = zip(
        *Xtest.apply(
            lambda x: explain_row(rf, x, num_reasons), 
            axis = 1, raw = True
        )
    )
    
    # Return the results
    for n in range(num_reasons):
        pred_ex['REASON%d' % (n+1)] = next(cols)
    return pred_ex


### Explanations for top cases predicted to belong to class 0

In [8]:
%%time
pe0l = predict_explain(clf[0], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 41.2 s


In [9]:
pe0l

,SCORE,REASON1,REASON2,TRUE_CLASS
53,1.00,proline > 996.25,flavanoids > 2.70,0
0,1.00,proline > 996.25,flavanoids > 2.70,0
56,1.00,flavanoids > 2.70,alcohol > 13.69,0
3,1.00,proline > 996.25,flavanoids > 2.70,0
18,1.00,proline > 996.25,flavanoids > 2.70,0
15,1.00,proline > 996.25,flavanoids > 2.70,0
5,1.00,proline > 996.25,flavanoids > 2.70,0
8,0.99,proline > 996.25,flavanoids > 2.70,0
42,0.98,proline > 996.25,flavanoids > 2.70,0
47,0.98,flavanoids > 2.70,total_phenols > 2.74,0


### Explanations for top cases predicted to belong to class 1

In [10]:
%%time
pe1l = predict_explain(clf[1], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 43 s


In [11]:
pe1l

,SCORE,REASON1,REASON2,TRUE_CLASS
87,1.00,color_intensity <= 3.14,alcohol <= 12.37,1
86,1.00,color_intensity <= 3.14,alcohol <= 12.37,1
113,1.00,color_intensity <= 3.14,alcohol <= 12.37,1
82,1.00,color_intensity <= 3.14,alcohol <= 12.37,1
117,0.99,color_intensity <= 3.14,proline <= 507.50,1
94,0.99,alcohol <= 12.37,proline <= 507.50,1
99,0.99,color_intensity <= 3.14,alcohol <= 12.37,1
119,0.99,color_intensity <= 3.14,alcohol <= 12.37,1
101,0.99,color_intensity <= 3.14,malic_acid <= 1.56,1
108,0.99,color_intensity <= 3.14,alcohol <= 12.37,1


### Explanations for top cases predicted to belong to class 2

In [12]:
%%time
pe2l = predict_explain(clf[2], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 40.3 s


In [13]:
pe2l

,SCORE,REASON1,REASON2,TRUE_CLASS
167,1.00,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
176,0.99,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
156,0.98,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
173,0.98,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
174,0.98,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
163,0.94,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
170,0.86,hue <= 0.74,flavanoids <= 1.19,2
133,0.84,od280/od315_of_diluted_wines <= 1.82,malic_acid > 2.92,2
154,0.84,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2
150,0.84,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,2


## Explanations by tree interpretation

In [14]:
import tree_explainer

### Explanations for top cases predicted to belong to class 0

In [15]:
%%time
pe0t = tree_explainer.predict_explain(clf[0], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 262 ms


In [16]:
pe0t

,SCORE,REASON1,REASON2,TRUE_CLASS
53,1.00,proline > 976.00,flavanoids > 2.61,0
0,1.00,proline > 976.00,flavanoids > 2.61,0
56,1.00,proline > 963.50,flavanoids > 2.93,0
3,1.00,proline > 1003.00,flavanoids > 2.93,0
18,1.00,proline > 976.00,flavanoids > 2.61,0
15,1.00,proline > 976.00,flavanoids > 2.61,0
5,1.00,proline > 976.00,flavanoids > 2.61,0
8,0.99,proline > 976.00,flavanoids > 2.61,0
42,0.98,proline > 1003.00,flavanoids > 2.93,0
47,0.98,proline > 983.00,flavanoids > 2.93,0


### Explanations for top cases predicted to belong to class 1

In [17]:
%%time
pe1t = tree_explainer.predict_explain(clf[1], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 267 ms


In [18]:
pe1t

,SCORE,REASON1,REASON2,TRUE_CLASS
87,1.00,color_intensity <= 3.38,alcohol <= 12.15,1
86,1.00,color_intensity <= 3.38,alcohol <= 12.23,1
113,1.00,color_intensity <= 3.38,alcohol <= 12.44,1
82,1.00,color_intensity <= 3.38,alcohol <= 12.44,1
117,0.99,color_intensity <= 3.14,alcohol <= 12.44,1
94,0.99,color_intensity <= 3.38,alcohol <= 12.23,1
99,0.99,color_intensity <= 3.38,alcohol <= 12.44,1
119,0.99,color_intensity <= 3.38,alcohol <= 12.06,1
101,0.99,color_intensity <= 3.09,alcohol <= 12.72,1
108,0.99,color_intensity <= 3.41,alcohol <= 12.23,1


### Explanations for top cases predicted to belong to class 2

In [19]:
%%time
pe2t = tree_explainer.predict_explain(clf[2], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 240 ms


In [20]:
pe2t

,SCORE,REASON1,REASON2,TRUE_CLASS
167,1.00,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74,2
176,0.99,od280/od315_of_diluted_wines <= 1.78,hue <= 0.65,2
156,0.98,od280/od315_of_diluted_wines <= 1.78,hue <= 0.65,2
173,0.98,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74,2
174,0.98,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74,2
163,0.94,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74,2
170,0.86,hue <= 0.74,od280/od315_of_diluted_wines <= 1.88,2
133,0.84,od280/od315_of_diluted_wines <= 1.48,hue <= 0.78,2
154,0.84,hue <= 0.65,od280/od315_of_diluted_wines <= 1.59,2
150,0.84,od280/od315_of_diluted_wines <= 1.53,hue <= 0.63,2


## Comparing the systems

### For class 0

In [21]:
pe0l[['REASON1','REASON2']].join(
    pe0t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
53,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
0,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
56,flavanoids > 2.70,alcohol > 13.69,proline > 963.50,flavanoids > 2.93
3,proline > 996.25,flavanoids > 2.70,proline > 1003.00,flavanoids > 2.93
18,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
15,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
5,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
8,proline > 996.25,flavanoids > 2.70,proline > 976.00,flavanoids > 2.61
42,proline > 996.25,flavanoids > 2.70,proline > 1003.00,flavanoids > 2.93
47,flavanoids > 2.70,total_phenols > 2.74,proline > 983.00,flavanoids > 2.93


### For class 1

In [22]:
pe1l[['REASON1','REASON2']].join(
    pe1t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
87,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.15
86,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.23
113,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.44
82,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.44
117,color_intensity <= 3.14,proline <= 507.50,color_intensity <= 3.14,alcohol <= 12.44
94,alcohol <= 12.37,proline <= 507.50,color_intensity <= 3.38,alcohol <= 12.23
99,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.44
119,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.38,alcohol <= 12.06
101,color_intensity <= 3.14,malic_acid <= 1.56,color_intensity <= 3.09,alcohol <= 12.72
108,color_intensity <= 3.14,alcohol <= 12.37,color_intensity <= 3.41,alcohol <= 12.23


### For class 2

In [23]:
pe2l[['REASON1','REASON2']].join(
    pe2t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
167,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74
176,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.65
156,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.65
173,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74
174,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74
163,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.78,hue <= 0.74
170,hue <= 0.74,flavanoids <= 1.19,hue <= 0.74,od280/od315_of_diluted_wines <= 1.88
133,od280/od315_of_diluted_wines <= 1.82,malic_acid > 2.92,od280/od315_of_diluted_wines <= 1.48,hue <= 0.78
154,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,hue <= 0.65,od280/od315_of_diluted_wines <= 1.59
150,od280/od315_of_diluted_wines <= 1.82,hue <= 0.74,od280/od315_of_diluted_wines <= 1.53,hue <= 0.63
